In [1]:
import os
import folium
import json
import math
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from PIL import Image

# Define full bounding box
LAT_MIN, LAT_MAX = 28.462, 28.723
LON_MIN, LON_MAX = 77.0592, 77.355

ROWS, COLS = 30, 30           # Total small tiles
BIG_ROWS, BIG_COLS = 6, 6     # Large screenshot grid
TILES_PER_BIG = 5             # 5x5 per large tile

LARGE_IMAGE_SIZE = 800        # For Selenium window
SMALL_TILE_SIZE = 32          # Output tile size
# SMALL_TILE_SIZE = 64          # Output tile size

OUTPUT_DIR = f"tiles_{SMALL_TILE_SIZE}x{SMALL_TILE_SIZE}"
BIG_DIR = "large_tiles"
META_FILE = "tile_metadata.json"

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(BIG_DIR, exist_ok=True)

# Simple lat/lon step size
lat_step = (LAT_MAX - LAT_MIN) / ROWS
lon_step = (LON_MAX - LON_MIN) / COLS

large_tile_metadata = {}

In [17]:
def get_bbox(row, col):
    lat1 = LAT_MIN + row * lat_step
    lat2 = lat1 + lat_step
    lon1 = LON_MIN + col * lon_step
    lon2 = lon1 + lon_step
    return [lat1, lon1, lat2, lon2]


In [3]:
# def create_map(lat1, lon1, lat2, lon2):
#     center = [(lat1 + lat2) / 2, (lon1 + lon2) / 2]
#     m = folium.Map(location=center, zoom_start=13, tiles=None, control_scale=False, zoom_control=False)
#     folium.TileLayer(
#         tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
#         attr="Esri",
#         name="Esri Satellite",
#         overlay=False,
#         control=False,
#     ).add_to(m)
#     m.fit_bounds([[lat1, lon1], [lat2, lon2]])
#     return m

def create_map(lat1, lon1, lat2, lon2):
    center = [(lat1 + lat2) / 2, (lon1 + lon2) / 2]
    m = folium.Map(location=center, zoom_start=13, tiles=None, control_scale=False, zoom_control=False)
    folium.TileLayer(
        tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
        attr="Esri",
        name="Esri Satellite",
        overlay=False,
        control=False,
    ).add_to(m)
    # Add a red border rectangle (1px wide)
    folium.Rectangle(
        bounds=[[lat1, lon1], [lat2, lon2]],
        color="#ff0000",
        weight=1,
        fill=False,
    ).add_to(m)
    m.fit_bounds([[lat1, lon1], [lat2, lon2]])
    return m

def crop_to_red_box(image: Image.Image):
    """Find the outer red rectangle (solid 2px border) and crop the inner content."""
    pixels = image.load()
    w, h = image.size

    min_x, min_y = w, h
    max_x, max_y = 0, 0

    def is_solid_red(r, g, b):
        return r > 200 and g < 80 and b < 80

    for x in range(w):
        for y in range(h):
            r, g, b = pixels[x, y][:3]
            if is_solid_red(r, g, b):
                min_x = min(min_x, x)
                max_x = max(max_x, x)
                min_y = min(min_y, y)
                max_y = max(max_y, y)

    if min_x >= max_x or min_y >= max_y:
        raise ValueError("Red box not found – try relaxing color thresholds.")

    # Add padding to exclude the red border (crop inside the box)
    crop_buffer = 2  # Skip the 2px red rectangle
    return image.crop((
        min_x + crop_buffer,
        min_y + crop_buffer,
        max_x - crop_buffer,
        max_y - crop_buffer
    ))


In [19]:
def setup_driver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument(f"--window-size={LARGE_IMAGE_SIZE},{LARGE_IMAGE_SIZE}")
    driver = webdriver.Chrome(options=options)
    return driver

In [20]:
metadata = {}
driver = setup_driver()

tile_idx = 0  # Keeps track of global row/col

for big_row in tqdm(range(BIG_ROWS), desc="Large Tiles"):
    for big_col in range(BIG_COLS):
        # Compute bounding box for this large tile (5x5 area)
        # lat1, lon1 = get_bbox(big_row * TILES_PER_BIG, big_col * TILES_PER_BIG)[:2]
        # lat2, lon2 = get_bbox((big_row+1) * TILES_PER_BIG - 1, (big_col+1) * TILES_PER_BIG - 1)[2:]
        # Compute precise bounding box for large tile (5x5 area)
        start_row = big_row * TILES_PER_BIG
        end_row = (big_row + 1) * TILES_PER_BIG
        start_col = big_col * TILES_PER_BIG
        end_col = (big_col + 1) * TILES_PER_BIG

        lat1 = LAT_MIN + start_row * lat_step
        lat2 = LAT_MIN + end_row * lat_step
        lon1 = LON_MIN + start_col * lon_step
        lon2 = LON_MIN + end_col * lon_step

        
        # Create map and save to HTML
        m = create_map(lat1, lon1, lat2, lon2)
        html_file = f"temp_map.html"
        m.save(html_file)
        
        # Load and screenshot
        driver.get("file://" + os.path.abspath(html_file))
        time.sleep(2)
        screenshot_file = os.path.join(BIG_DIR, f"large_{big_row}_{big_col}.png")
        driver.save_screenshot(screenshot_file)

        # Crop and split into 5x5 small tiles
        img = Image.open(screenshot_file)
        img = crop_to_red_box(img)  # Crop using red box

        w, h = img.size
        tile_w = w // TILES_PER_BIG
        tile_h = h // TILES_PER_BIG
        
        for i in range(TILES_PER_BIG):
            for j in range(TILES_PER_BIG):
                crop_box = (
                    j * tile_w,
                    i * tile_h,
                    (j + 1) * tile_w,
                    (i + 1) * tile_h,
                )
                tile = img.crop(crop_box).resize((SMALL_TILE_SIZE, SMALL_TILE_SIZE), Image.Resampling.LANCZOS)

                row = big_row * TILES_PER_BIG + (TILES_PER_BIG - i - 1)
                col = big_col * TILES_PER_BIG + j
                tile_name = f"tile_{row}_{col}.png"
                tile.save(os.path.join(OUTPUT_DIR, tile_name))

                tile_lat1, tile_lon1, tile_lat2, tile_lon2 = get_bbox(row, col)
                metadata[f"{row}_{col}"] = {
                    "row": row,
                    "col": col,
                    "center": [ (tile_lat1 + tile_lat2)/2, (tile_lon1 + tile_lon2)/2 ],
                    "bbox": [tile_lat1, tile_lon1, tile_lat2, tile_lon2]
                }

# Final full-region screenshot
print("Taking full-region screenshot...")
full_map = create_map(LAT_MIN, LON_MIN, LAT_MAX, LON_MAX)
full_map.save("temp_full.html")
driver.get("file://" + os.path.abspath("temp_full.html"))
time.sleep(3)
driver.save_screenshot("full_region.png")

driver.quit()
with open(META_FILE, "w") as f:
    json.dump(metadata, f, indent=2)

print("Done! All tiles and metadata saved.")

Large Tiles: 100%|██████████| 6/6 [01:59<00:00, 19.96s/it]


Taking full-region screenshot...
Done! All tiles and metadata saved.


In [21]:
import os
import folium
import json
import math
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from PIL import Image

In [22]:
with open("tile_metadata.json", "r") as f:
    metadata = json.load(f)

In [23]:
metadata["15_10"]

{'row': 15,
 'col': 10,
 'center': [28.596850000000003, 77.16273000000001],
 'bbox': [28.5925, 77.15780000000001, 28.601200000000002, 77.16766000000001]}

In [4]:
img = Image.open('./full_region.png')

img = crop_to_red_box(img)

In [6]:
img.save("full_region_cropped.png")